In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook, tnrange

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [2]:
df = pd.read_csv('test.csv/test.csv', index_col='id', skiprows=range(44000, 49501))

In [3]:
%load_ext Cython

In [4]:
%%cython 

cimport cython
import numpy as np

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cdef int calc_neighs(unsigned char[:, :] field, int i, int j, int n):
    cdef:
        int neighs = 0;
        int i_min = i - 1;
        int i_pl = i + 1;
        int j_min = j - 1;
        int j_pl = j + 1;
        int k, row_idx, col_idx;
    neighs = 0
    if i_min >= 0:
        if j_min >= 0:
            neighs += field[i_min, j_min]
        neighs += field[i_min, j]
        if j_pl < n:
            neighs += field[i_min, j_pl]
    if j_min >= 0:
        neighs += field[i, j_min]
    if j_pl < n:
        neighs += field[i, j_pl]
    if i + 1 < n:
        if j_min >= 0:
            neighs += field[i_pl, j_min]
        neighs += field[i_pl, j]
        if j_pl < n:
            neighs += field[i_pl, j_pl]
    return neighs

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef make_move(unsigned char[:, :] field, int moves):
    cdef:
        int _, i, j, neighs;
        int n;
        int switch = 0;
        unsigned char[:, :] cur_field;
        unsigned char[:, :] next_field;
    cur_field = np.copy(field)
    next_field = np.zeros_like(field, 'uint8')
    n = len(field)
    for _ in range(moves):
        if switch == 0:
            for i in range(n):
                for j in range(n):
                    neighs = calc_neighs(cur_field, i, j, n)
                    if cur_field[i, j] and neighs == 2:
                        next_field[i, j] = 1
                    elif neighs == 3:
                        next_field[i, j] = 1
                    else:
                        next_field[i, j] = 0
        else:
            for i in range(n):
                for j in range(n):
                    neighs = calc_neighs(next_field, i, j, n)
                    if next_field[i, j] and neighs == 2:
                        cur_field[i, j] = 1
                    elif neighs == 3:
                        cur_field[i, j] = 1
                    else:
                        cur_field[i, j] = 0
        switch = (switch + 1) % 2
    return np.array(next_field if switch else cur_field)

In [5]:
import numpy as np
import multiprocessing as mp
from functools import partial

      
def parallel_fitness(gene, Y, delta):
    candidate = make_move(gene, moves=delta)
    return (candidate == Y).sum() / 400

class GeneticSolver:
    def __init__(self, population_size=800, n_generations=2000, retain_best=0.8, retain_random=0.05, mutate_chance=0.05,
                 verbosity=0, verbosity_step=20, random_state=-1, warm_start=False, early_stopping=True, patience=20,
                 initialization_strategy='uniform', fitness_parallel=False):
        """
        :param population_size: number of individual candidate solutions
        :param n_generations: number of generations
        :param retain_best: percentage of best candidates to select into the next generation
        :param retain_random: probability of selecting sub-optimal candidate into the next generation
        :param mutate_chance: candidate mutation chance
        :param verbosity: level of verbosity (0 - quiet, 1 - evolution information, 2 - spamming like in 2003)
        :param random_state: if specified, initializes seed with this value
        :param warm_start: if True, initial population generation step is omitted, allowing for continuing training
        :param early_stopping: if True, evolution will stop if top-10 candidates are not changing for several generations
        :param patience: number of generations to wait for best solution change when <early_stopping>
        :param initialization_strategy: initial population generation rule: 'uniform' or 'covering'
        """
        self.population_size = population_size
        self.n_generations = n_generations
        self.retain_best = retain_best
        self.retain_random = retain_random
        self.mutate_chance = mutate_chance
        self.verbosity = verbosity
        self.verbosity_step = verbosity_step
        self.random_state = random_state
        self.warm_start = warm_start
        self.early_stopping = early_stopping
        self.patience = patience
        self.initialization_strategy = initialization_strategy
        self.fitness_parallel = fitness_parallel
        if fitness_parallel:
            self.pool = mp.Pool(mp.cpu_count())
        else:
            self.pool = None

        self._population = None
        if random_state != -1:
            np.random.seed(random_state)
            
    def set_position(self, position):
        self.position = position # number in pool

    def solve(self, Y, delta, n_generations=-1):
        """

        :param Y: 20x20 array that represents field in stopping condition
        :param delta: number of steps to revert
        :param n_generations: number of evolution generations. Overrides initialization value if specified
        :return: 20x20 array that represents the best start field found and associated fitness value
        """
        if not (self._population and self.warm_start):
            self._population = self._generate_population()
        if n_generations != -1:
            self.n_generations = n_generations
        scores = np.zeros(len(self._population))
        prev_scores = np.zeros(len(self._population))
        cnt_no_change_in_scores = 0
        for generation in tnrange(self.n_generations, desc="evolution"+str(self.position), position=self.position + 1, leave=False):
            self._population, scores = self.evolve(Y, delta)
            if np.isclose(prev_scores[:10], scores[:10]).all():
                cnt_no_change_in_scores += 1
            else:
                cnt_no_change_in_scores = 0
                prev_scores = scores
            if self.verbosity and generation % self.verbosity_step == 0:
                if generation == 0:
                    print(f"Generation #: best score")
                else:
                    print(f"Generation {generation}: {scores[0]}")
            if np.isclose(scores[:10], 1).any() or (self.early_stopping and cnt_no_change_in_scores >= self.patience):
                if self.verbosity:
                    print(f"Early stopping on generation {generation} with best score {scores[0]}")
                break
        return self._population[0], scores[0]

    def _generate_population(self):
        """
        Generating initial population of individual solutions

        Regardless of strategy, we make 5 initial "warming" steps to make distribution closer to the problem.

        Strategies description:

            * Uniform: each cell has equal probability of being initialized as alive or dead. This will introduce no
                       prior information at all
            * Covering: Each individual is generated with it's own probability of having each cell 'alive'. This gives
                       on average higher initial fitness score, but has no observed effect on long-term behavior
        :return: initial population as a list of 20x20 arrays
        """
        if self.initialization_strategy == 'uniform':
            initial_states = np.split(np.random.binomial(1, 0.5, (20 * self.population_size, 20)).astype('uint8'), self.population_size)
            return [make_move(state, 5) for state in initial_states]
        elif self.initialization_strategy == 'covering':
            """ Idea is to cover all the range of possible values for 'density' parameter """
            alive_probabilities = np.linspace(0.01, 0.99, self.population_size)
            return [make_move(np.random.binomial(1, prob, size=(20, 20)), moves=5) for prob in alive_probabilities]
        else:
            raise NotImplementedError(f"{self.initialization_strategy} is not implemented!")

    def evolve(self, Y, delta):
        """
        Evolution step
        :param Y: 20x20 array that represents field in stopping condition
        :param delta: number of steps to revert
        :return: new generation of the same size along with scores of the best retained individuals
        """
        if self.fitness_parallel:
          scores = np.array(self.parallel_score_population(self._population, Y, delta))
        else:
          scores = np.array(self.score_population(self._population, Y, delta))
        retain_len = int(len(scores) * self.retain_best)
        sorted_indices = np.argsort(scores)[::-1]
        self._population = [self._population[idx] for idx in sorted_indices]
        best_scores = scores[sorted_indices][:retain_len]
        if self.verbosity > 1:
            print("best scores:", best_scores)
        parents = self._population[:retain_len]
        leftovers = self._population[retain_len:]

        cnt_degenerate = 0
        for gene in leftovers:
            if np.random.rand() < self.retain_random:
                cnt_degenerate += 1
                parents.append(gene)
        if self.verbosity > 1:
            print(f"# of degenerates left: {cnt_degenerate}")

        cnt_mutations = 0
        for gene in parents[1:]:  # mutate everyone expecting for the best candidate
            if np.random.rand() < self.mutate_chance:
                self.mutate(gene)
                cnt_mutations += 1
        if self.verbosity > 1:
            print(f"# of mutations: {cnt_mutations}")

        places_left = self.population_size - retain_len
        children = []
        while len(children) < places_left:
            mom_idx, dad_idx = np.random.randint(0, retain_len - 1, 2)
            if mom_idx != dad_idx:
                child1, child2 = self.crossover(parents[mom_idx], parents[dad_idx])
                children.append(child1)
                if len(children) < places_left:
                    children.append(child2)
        if self.verbosity > 1:
            print(f"# of children: {len(children)}")
        parents.extend(children)
        return parents, best_scores

    @classmethod
    def crossover(cls, mom, dad):
        """
        Take two parents, return two children, interchanging half of the allels of each parent randomly
        """
        # select_mask = np.random.randint(0, 2, size=(20, 20), dtype='bool')
        select_mask = np.random.binomial(1, 0.5, size=(20, 20)).astype('bool')
        child1, child2 = np.copy(mom), np.copy(dad)
        child1[select_mask] = dad[select_mask]
        child2[select_mask] = mom[select_mask]
        return child1, child2

    @classmethod
    def mutate(cls, field):
        """
        Inplace mutation of the provided field
        """
        a = np.random.binomial(1, 0.1, size=(20, 20)).astype('bool')
        field[a] += 1
        field[a] %= 2
        return field

    @classmethod
    def fitness(cls, start_field, end_field, delta):
        """
        Calculate fitness for particular candidate (start configuration of the field)
        :param start_field: candidate (start configuration)
        :param end_field: target (stop configuration)
        :param delta: number of steps to proceed before comparing to stop configuration
        :return: value in range [0, 1] that indicates fractions of cells that match their state
        """
        candidate = make_move(start_field, moves=delta)
        return (candidate == end_field).sum() / 400

      
    @classmethod
    def score_population(cls, population, Y, delta):
        """
        Apply fitness function for each gene in a population
        :param population: list of candidate solutions
        :param Y: 20x20 array that represents field in stopping condition
        :param delta: number of steps to revert
        :return: list of scores for each solution
        """
        return [cls.fitness(gene, Y, delta) for gene in population]

    def parallel_score_population(self, population, Y, delta):
        """
        Apply fitness function for each gene in a population in parallel
        :param population: list of candidate solutions
        :param Y: 20x20 array that represents field in stopping condition
        :param delta: number of steps to revert
        :return: list of scores for each solution
        """
        return self.pool.map(partial(parallel_fitness, Y=Y, delta=delta), population)


In [6]:
import multiprocessing as mp
import scipy


def work(solver, Y, delta):
    # this is required for every worker to have different initial seed. Otherwise they inherit it from this thread
    scipy.random.seed()
    return solver.solve(Y, delta)


class MPGeneticSolver:
    def __init__(self, n_proc='auto', *args, **kwargs):
        """
        Multi-process version of Genetic Solver with different initial conditions
        :param n_proc: number of processes to create
        :param args: GeneticSolver arguments (see its documentation for more)
        :param kwargs: GeneticSolver key-value arguments
        """
        if n_proc == 'auto':
            n_proc = mp.cpu_count()
        self.n_proc = n_proc
        self.pool = mp.Pool(mp.cpu_count() if n_proc == 'auto' else n_proc)
        self.args = args
        self.kwargs = kwargs
        self._solvers = None
        if 'fitness_parallel' in self.args or ('fitness_parallel' in self.kwargs and self.kwargs['fitness_parallel']):
            raise ValueError("Fitness function cannot be parallelized in MPGeneticSolver")

    def solve(self, Y, delta, return_all=True):
        """
        Solve RGoL problem
        :param Y: 20x20 array that represents field in stopping condition
        :param delta: number of steps to revert
        :param return_all: if True, returns all of the results from different runners, as well as their scores.
                           If False only solution associated with the best score is returned
        :return: either list of (solution, score) pairs or the best solution (see `return_all`)
        """
        self._solvers = [GeneticSolver(*self.args, **self.kwargs) for _ in range(self.n_proc)]
        for position in range(self.n_proc):
            self._solvers[position].set_position(position)
        tasks = [(solver, Y, delta) for solver in self._solvers]
        results = self.pool.starmap(work, tasks)
        return results if return_all else self.select_best(results)

    @classmethod
    def select_best(cls, solutions):
        """
        Using output of solve method, select the best solution
        :param solutions: list of (solution, score) pairs
        :return: 20x20 array that represents the solution (starting board condition)
        """
        return sorted(solutions, key=lambda x:x[1], reverse=True)[0]

In [7]:
class SolutionRunner:
  def __init__(self, save_fname='solution.csv', verbosity=0):
    self.save_fname = save_fname
    self.verbosity = verbosity
    self.log = []
    self.running_avg = 0
    self.n = 0
  
  
  def solve_df(self, df, first_n=None, save_to=None):
    solver = MPGeneticSolver(early_stopping=False, n_proc=4)
    
    solution_df = pd.DataFrame([], columns=['id', 'score'] + ['start.'+ str(_) for _ in range(1, 401)], dtype=int)
    for col in solution_df.columns:
      solution_df[col] = solution_df[col].astype(np.int32)
    
    self.running_avg = 0
    self.n = 0
    self.log = []
    best, worst = None, None
    with tqdm_notebook(total=5500, position=0) as pbar:
        for i, (id, (idx, row)) in enumerate(zip(df.index, df.iterrows())):
            delta, Y = row.values[0], row.values[1:].reshape((20, 20)).astype('uint8')
            solution = solver.solve(Y, delta, return_all=False)

            board, score = solution
            flat_board = np.insert(board.ravel(), 0, id)
            flat_board = np.insert(flat_board, 1, int(score * 100))
            solution_df = solution_df.append(pd.Series(flat_board, index=solution_df.columns), ignore_index=True)

            self.log.append((idx, score))
            if best is None or best[1] < score:
                best = (idx, score)
            if worst is None or worst[1] > score:
                worst = (idx, score)
            self.n += 1
            self.running_avg = (self.running_avg * (self.n - 1) + score) / self.n
            if self.verbosity:
              print(f"{idx} is solved with score {score}. Average score: {self.running_avg}")
            if first_n and i >= first_n:
              break
            pbar.set_postfix({"idx": idx, "score": score, "av score": self.running_avg})
            pbar.update()
    if self.verbosity:
      print("Best score:", best)
      print("Worst score:", worst)
    if save_to is not None:
      solution_df.to_csv(save_to, index=False)
    else:
      solution_df.to_csv(self.save_fname, index=False)
    return solution_df

In [8]:
sr = SolutionRunner(verbosity=0)

In [9]:
solution = sr.solve_df(df, 5500, 's44000-49501.csv')